In [3]:
%load_ext autoreload
%autoreload 10

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import cv2
import dlib
import imutils
import numpy as np

from tqdm import tqdm

In [5]:
import os
import sys
import glob

sys.path.append("../")

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [15]:
from facelib.transformers import crop_face_bounding_box

In [7]:
IMAGETYPES = ('*.bmp', '*.png', '*.jpg', '*.jpeg', '*.tif') # Supported image types


def get_image_paths(file_root, pattern=None):
    """ Get ordered list of filepaths
    """
    file_paths = []
    for typ in IMAGETYPES:
        file_paths.extend(glob.glob(os.path.join(file_root,'**',typ), recursive=True))

    # filter filenames
    if pattern is not None:
        ffiltered = []
        ffiltered = [f for f in file_paths if pattern in os.path.split(f)[-1]]
        file_paths = ffiltered
        del ffiltered

    file_paths.sort()
    return file_paths


def load_face_dataset(file_root):
    """ Load all images and also list out the filepaths
    """
    if not os.path.isdir(file_root):
        raise FileNotFoundError("Face dataset file root: {} not found.".format(file_root))

    file_paths = get_image_paths(file_root)
    
    if len(file_paths) == 0:
        warnings.warn("No image found in face dataset file root: {}".format(file_root))
        return []
    
    images = [cv2.imread(file_path) for file_path in file_paths]
    return images, file_paths

In [20]:
""" Definition for FaceDetector
"""

imageMean = [104.0, 177.0, 123.0]
confThresh = 0.5


class FaceDetector(BaseEstimator, TransformerMixin):
    """ Face dataset transformers that detect faces in an image and yield cropped faces image
    """

    def __init__(self, prototxt, caffeModel, imsize=(300,300), 
                 multiFaceMode=False, strict=False):
        """ Initialise face detector

        Args:
            .. prototxt   (str/path): path to prototxt file defining the architecture
            .. caffeModel (str/path): path to checkpoint file of pretrained caffe model
            .. imsize   (Tuple[int]): height and width of image blob
            .. multiFaceMode  (bool): whether to detect only multiple or single face for each image
            .. strict (bool) : if no face detected, raise error
        """
        if not os.path.isfile(prototxt):
            raise FileNotFoundError("Prototxt file: {} not found.".format(prototxt))
        
        if not os.path.isfile(caffeModel):
            raise FileNotFoundError("CaffeModel file: {} not found.".format(caffeModel))

        self.imsize = imsize
        self.multiFaceMode = multiFaceMode
        self.detector = cv2.dnn.readNetFromCaffe(prototxt, caffeModel=caffeModel)

    def fit(self, *args, **kwargs):
        # unused but need to satisfy abstraction rule
        return self

    def transform(self, images):
        """ Perform face detection and cropping for batch of images
        
        Args:
            .. images (np.ndarray): array of images in tensor form of shape [C,H,W]

        Returns:
            .. faces (np.ndarray or List[List[np.ndarray]]): array of images in numpy tensor.
                    If multiFaceMode false, output is tensor of shape: [N,C,H,W]
                    If multiFaceMode true, output is array of cropped faces.
        """
        if self.multiFaceMode == True:
            rects = [self._detect_multi_faces(image) for image in tqdm(images)]
        else:
            rects = np.stack([self._detect_single_face(image) for image in tqdm(images)])
        return images, rects

    # -------------------
    # Detector helper
    # -------------------

    def _detect_single_face(self, image):
        """ Apply detection assuming only one face per image
        
        Args:
            .. image (np.ndarray): images in numpy tensor form of shape [C,H,W]
        
        Returns:
            .. face (np.ndarray or None): image in numpy tensor of shape [C,H,W]. None if no face
        """
        h, w = image.shape[:2]

        # construct a blob from the image
        imageBlob = cv2.dnn.blobFromImage(cv2.resize(image, self.imsize), 1.0, self.imsize, imageMean)

        # apply OpenCV's deep learning-based face detector to localize
        # faces in the input image
        self.detector.setInput(imageBlob)
        detections = self.detector.forward()
        
        # ensure at least one face was found
        if len(detections) > 0:
            # we're making the assumption that each image has only ONE
            # face, so find the bounding box with the largest probability
            i = np.argmax(detections[0, 0, :, 2])
            confidence = detections[0, 0, i, 2]

            # filter out weak detections
            if confidence > confThresh:
                # compute the (x, y)-coordinates of the bounding box for
                # the face
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # grab the ROI dimensions
                (fH, fW) = endY - startY, endX - startX

                # ensure the face width and height are sufficiently large
                if fW > 40 and fH > 40:
                    rect = dlib.rectangle(startX, startY, startX+fW, startY+fH)
                    return rect

        if strict:
            raise ValueError("Unable to detect face during FaceDetection in one of the image passed.\n"
                             "Ensure that image is clean and has face in it. "
                             "You can also \n increase the threshold.")
        return None
    
    def _detect_multi_faces(self, image):
        """ Apply detection assuming only one face per image
        
        Args:
            .. image (np.ndarray): images in numpy tensor form of shape [C,H,W]
        
        Returns:
            .. face_images (np.ndarray or None): array of faces images of shape 
        """
        blob = cv2.dnn.blobFromImage(cv2.resize(image, self.imsize, 1.0, self.imsize, imageMean))

        # apply OpenCV's deep learning-based face detector to localize
        # faces in the input image
        self.detector.setInput(imageBlob)
        detections = self.detector.forward()

        rects = []
        
        # loop over the detections
        for i in range(0, detections.shape[2]):
            # extract the confidence (i.e., probability) associated with
            # the prediction
            confidence = detections[0, 0, i, 2]

            # filter out weak detections with less than 50% confidence
            if confidence > 0.5:
                # compute the (x, y)-coordinates of the bounding box for
                # the face
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # grab the ROI dimensions
                (fH, fW) = endY - startY, endX - startX

                # ensure the face width and height are sufficiently large
                if fW < 40 or fH < 40:
                    continue

                rect = dlib.rectangle(startX, startY, startX+fW, startY+fH)
                rects.append(rect)

        return rects

In [16]:
data_dir = "../data/temp/"

images, file_paths = load_face_dataset(data_dir)

### Simple Pipeline 

In [17]:
# define file paths to model files
prototxt = '../models/deploy.prototxt'
caffeModel = '../models/res10_300x300_ssd_iter_140000.caffemodel'

In [ ]:
pipe1 = Pipeline([
    ('facedetector', FaceDetector(prototxt, caffeModel)),
    ('cropface', crop_face_bounding_box())
])

In [ ]:
results = pipe1.fit_transform(images)